In [36]:
import pprint
pp = pprint.PrettyPrinter(indent=4)
!pip install tensorflow
!pip install tfx
import tensorflow
import tfx

3642.34s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


3652.82s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


  Using cached tfx-0.23.1-py3-none-any.whl (1.7 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 27.0 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 13.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.9/326.9 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.1/172.1 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.7/125.7 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 4.2 MB/s eta 0:00:0000:01:00:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 2.6 MB/s eta 0:00:0000:01:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 4.6 MB/s eta 0:00:0000:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 34.2 MB/s eta 0:00:00:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 

2023-03-30 14:58:04.042146: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-03-30 14:58:06.545758: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-03-30 14:58:06.558645: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-30 14:58:10.512310: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


ModuleNotFoundError: No module named 'tfx'

In [15]:
'''
pipeline = tfx.dsl.Pipeline(
    pipeline_name='{pipeline_name}',
    pipeline_root='{pipeline_root}',
    components=[
        {example_gen},
        {statistics_gen},
        {schema_gen},
        {example_validator},
        {transform},
        {trainer},
        {evaluator},
        {pusher}
    ]
)
'''

"\npipeline = tfx.dsl.Pipeline(\n    pipeline_name='{pipeline_name}',\n    pipeline_root='{pipeline_root}',\n    components=[\n        {example_gen},\n        {statistics_gen},\n        {schema_gen},\n        {example_validator},\n        {transform},\n        {trainer},\n        {evaluator},\n        {pusher}\n    ]\n)\n"

In [6]:
# Define the component configurations
input_path = 'path/to/your/input_data'
module_path = 'path/to/your/module'
serving_path = 'path/to/your/serving_model'

component_configs = {
    'example_gen': f'example_gen = tfx.components.CsvExampleGen(input_base="{input_path}")',
    'statistics_gen': 'statistics_gen = tfx.components.StatisticsGen(examples=example_gen.outputs.examples)',
    'schema_gen': 'schema_gen = tfx.components.SchemaGen(statistics=statistics_gen.outputs.statistics)',
    'example_validator': 'example_validator = tfx.components.ExampleValidator(statistics=statistics_gen.outputs.statistics, schema=schema_gen.outputs.schema)',
    'transform': f'transform = tfx.components.Transform(examples=example_gen.outputs.examples, schema=schema_gen.outputs.schema, module_file="{module_path}")',
    'trainer': f'''trainer = tfx.components.Trainer(
        module_file="{module_path}",
        transformed_examples=transform.outputs.transformed_examples,
        schema=schema_gen.outputs.schema,
        transform_graph=transform.outputs.transform_graph,
        train_args=tfx.proto.TrainArgs(num_steps=1000),
        eval_args=tfx.proto.EvalArgs(num_steps=500)
    )''',
    'evaluator': 'evaluator = tfx.components.Evaluator(examples=example_gen.outputs.examples, model=trainer.outputs.model, schema=schema_gen.outputs.schema)',
    'pusher': f'pusher = tfx.components.Pusher(model_export=trainer.outputs.model, serving_model_dir="{serving_path}")'
}


In [13]:
print(component_configs)
print(type(component_configs))
print(component_configs['pusher'])
print(component_configs['schema_gen'])
print(type(component_configs['schema_gen']))

{'example_gen': 'example_gen = tfx.components.CsvExampleGen(input_base="path/to/your/input_data")', 'statistics_gen': 'statistics_gen = tfx.components.StatisticsGen(examples=example_gen.outputs.examples)', 'schema_gen': 'schema_gen = tfx.components.SchemaGen(statistics=statistics_gen.outputs.statistics)', 'example_validator': 'example_validator = tfx.components.ExampleValidator(statistics=statistics_gen.outputs.statistics, schema=schema_gen.outputs.schema)', 'transform': 'transform = tfx.components.Transform(examples=example_gen.outputs.examples, schema=schema_gen.outputs.schema, module_file="path/to/your/module")', 'trainer': 'trainer = tfx.components.Trainer(\n        module_file="path/to/your/module",\n        transformed_examples=transform.outputs.transformed_examples,\n        schema=schema_gen.outputs.schema,\n        transform_graph=transform.outputs.transform_graph,\n        train_args=tfx.proto.TrainArgs(num_steps=1000),\n        eval_args=tfx.proto.EvalArgs(num_steps=500)\n  

In [20]:
# Define the pipeline configuration
pipeline_config = {
    'pipeline_name': 'my_pipeline',
    'pipeline_root': '/path/to/pipeline_root',
    'components': []
}

user_selected_components = ['example_gen', 'statistics_gen', 'schema_gen', 'trainer', 'pusher']

# Generate the component code and add to the pipeline configuration
for component_name in user_selected_components:
    component_config = component_configs[component_name]
    pipeline_config['components'].append(component_config)

pp.pprint(pipeline_config)

{   'components': [   'example_gen = '
                      'tfx.components.CsvExampleGen(input_base="path/to/your/input_data")',
                      'statistics_gen = '
                      'tfx.components.StatisticsGen(examples=example_gen.outputs.examples)',
                      'schema_gen = '
                      'tfx.components.SchemaGen(statistics=statistics_gen.outputs.statistics)',
                      'trainer = tfx.components.Trainer(\n'
                      '        module_file="path/to/your/module",\n'
                      '        '
                      'transformed_examples=transform.outputs.transformed_examples,\n'
                      '        schema=schema_gen.outputs.schema,\n'
                      '        '
                      'transform_graph=transform.outputs.transform_graph,\n'
                      '        '
                      'train_args=tfx.proto.TrainArgs(num_steps=1000),\n'
                      '        eval_args=tfx.proto.EvalArgs(num_st

In [ ]:
'''
Example output : 
import tensorflow as tf
import tfx

# Define a TFX pipeline
pipeline = tfx.dsl.Pipeline(
    pipeline_name='my_pipeline',
    pipeline_root='/path/to/pipeline_root',
    components=[
        example_gen,
        statistics_gen,
        schema_gen,
        example_validator,
        transform,
        trainer,
        evaluator,
        pusher
    ]
)

# Run the pipeline
tfx.orchestration.LocalDagRunner().run(pipeline)

'''

In [28]:
# Generate the pipeline code dynamically
pipeline_code = '''
pipeline = tfx.dsl.Pipeline(
    pipeline_name='{pipeline_name}',
    pipeline_root='{pipeline_root}',
    components=[
        {component_code}
    ]
)
'''.format(
    pipeline_name=pipeline_config['pipeline_name'],
    pipeline_root=pipeline_config['pipeline_root'],
    component_code=',\n'.join(pipeline_config['components'])
)

# Print the pipeline code
print(type(pipeline_code))
print(pipeline_code)

# # Execute the pipeline code
# exec(pipeline_code)


<class 'str'>

pipeline = tfx.dsl.Pipeline(
    pipeline_name='my_pipeline',
    pipeline_root='/path/to/pipeline_root',
    components=[
        example_gen = tfx.components.CsvExampleGen(input_base="path/to/your/input_data"),
statistics_gen = tfx.components.StatisticsGen(examples=example_gen.outputs.examples),
schema_gen = tfx.components.SchemaGen(statistics=statistics_gen.outputs.statistics),
trainer = tfx.components.Trainer(
        module_file="path/to/your/module",
        transformed_examples=transform.outputs.transformed_examples,
        schema=schema_gen.outputs.schema,
        transform_graph=transform.outputs.transform_graph,
        train_args=tfx.proto.TrainArgs(num_steps=1000),
        eval_args=tfx.proto.EvalArgs(num_steps=500)
    ),
pusher = tfx.components.Pusher(model_export=trainer.outputs.model, serving_model_dir="path/to/your/serving_model")
    ]
)



In [29]:
# Example to execute codes : 
ex_var = "name='Hello World' "
ex_str = "print(name)"

print(ex_var)
print(ex_str)

exec(ex_var)
exec(ex_str)

name='Hello World' 
print(name)
Hello World


In [32]:
# Using Abstract Syntax Tree 
import ast
import tfx.dsl.component.experimental.resolver as resolver
import tfx.dsl.experimental.assembly_utils as assembly_utils
import tfx.dsl.experimental.pipeline_template as pipeline_template
import tfx.orchestration.experimental.kubernetes.kube_executor as kube_executor

# Define the components of the pipeline as objects
example_gen = ...
statistics_gen = ...
schema_gen = ...
example_validator = ...
transform = ...
trainer = ...
evaluator = ...
pusher = ...

# Create an ast.Name object for each component object
example_gen_node = ast.Name(id='example_gen', ctx=ast.Load())
statistics_gen_node = ast.Name(id='statistics_gen', ctx=ast.Load())
schema_gen_node = ast.Name(id='schema_gen', ctx=ast.Load())
example_validator_node = ast.Name(id='example_validator', ctx=ast.Load())
transform_node = ast.Name(id='transform', ctx=ast.Load())
trainer_node = ast.Name(id='trainer', ctx=ast.Load())
evaluator_node = ast.Name(id='evaluator', ctx=ast.Load())
pusher_node = ast.Name(id='pusher', ctx=ast.Load())

# Create an ast.List object containing the component nodes
components_node = ast.List(
    elts=[example_gen_node, statistics_gen_node, schema_gen_node,
          example_validator_node, transform_node, trainer_node,
          evaluator_node, pusher_node],
    ctx=ast.Load()
)

# Create an ast.Dict object representing the pipeline configuration
pipeline_config_node = ast.Dict(
    keys=[ast.Str(s='pipeline_name'), ast.Str(s='pipeline_root'),
          ast.Str(s='components')],
    values=[ast.Str(s='my_pipeline'), ast.Str(s='/path/to/pipeline_root'),
            components_node]
)

# Create an ast.Call object representing the pipeline creation code
pipeline_call_node = ast.Call(
    func=ast.Attribute(
        value=ast.Name(id='tfx.dsl.Pipeline', ctx=ast.Load()),
        attr='from_static_pipeline_config', ctx=ast.Load()
    ),
    args=[pipeline_config_node],
    keywords=[]
)

# Create an ast.Assign object to assign the pipeline object to a variable
pipeline_assign_node = ast.Assign(
    targets=[ast.Name(id='pipeline', ctx=ast.Store())],
    value=pipeline_call_node
)

# Create an ast.Expr object representing the entire pipeline code
pipeline_expr_node = ast.Expr(value=pipeline_assign_node)

# Create an ast.Module object containing the pipeline expression
module_node = ast.Module(body=[pipeline_expr_node])

# Print the generated Python code
print(ast.dump(module_node))


ModuleNotFoundError: No module named 'tfx'

In [39]:
import ast
import astunparse

# Define the AST node for the function
func_node = ast.FunctionDef(
    name='square',
    args=ast.arguments(
        args=[ast.arg(arg='x', annotation=None)],
        vararg=None,
        kwonlyargs=[],
        kw_defaults=[],
        kwarg=None,
        defaults=[]
    ),
    body=[ast.Return(value=ast.BinOp(
        left=ast.Name(id='x', ctx=ast.Load()),
        op=ast.Mult(),
        right=ast.Name(id='x', ctx=ast.Load())
    ))],
    decorator_list=[],
    returns=None
)

# Generate the code from the AST node
func_code = astunparse.unparse(func_node)

# Compile the code
compiled_code = compile(func_code, '<string>', 'exec')

# Execute the code with x=10
x = 10
local_vars = {'x': x}
exec(compiled_code, None, local_vars)

# Call the function with x=10
result = local_vars['square'](x)
print(result) # Output: 100


100
